# CLIENT (SENDER)

In [39]:
import numpy as np
import binascii

In [40]:
def decimalToBinary(num , places):
    return ('{0:0' +  str(places) + 'b}').format(num)


def binaryToDecimal(binary):
    return int(binary, 2)
        
def xorBinary(binary,key = "00000000"):
    i=0
    encBin = ""
    while(i<8):
        if(binary[i] == '0' and key[i] == '0') or (binary[i] == '1' and key[i] == '1'):
            encBin+="0"
        else:
            encBin+="1"
        i+=1
        
#     print(binary , key , encBin)    
    return encBin
    
def compressBinary(binary):
    i=0
    compressBin = ""
    while(i<8):
        if(i%2 == 0):
            if((binary[i] == '0' and binary[i+1] == '0') or (binary[i] == '1' and binary[i+1] == '1')):
                compressBin+="0"
            else:
                compressBin+="1"
        i+=1
        
    return compressBin

def to_twoscomplement(bits, value):
    if value < 0:
        value = ( 1<<bits ) + value
    formatstring = '{:0%ib}' % bits
    return formatstring.format(value)

# Code for importing data

In [41]:
import pandas as pd
article_read = pd.read_csv('dataset.csv',delimiter = ',')

In [42]:
# new_dataset = pd.Series(list(article_read['Temp.'].sample(101 , random_state=10)))
new_dataset = article_read['Temp.']

# new_dataset

# Code for delta coding

In [43]:
new_dataset.describe()

count    928991.000000
mean         27.283373
std           0.904537
min          24.434400
25%          26.764900
50%          27.328400
75%          27.868700
max          30.325400
Name: Temp., dtype: float64

In [44]:
int_dataset = new_dataset * 10000

In [45]:
int_dataset = int_dataset.astype('int64')

In [46]:
int_dataset.head()

0    262257
1    262308
2    262365
3    262416
4    262462
Name: Temp., dtype: int64

In [47]:
diff_dataset = [int_dataset[0]]
for i in range(1 , len(int_dataset)) :
        diff_dataset.append(int_dataset[i] - int_dataset[i-1])

In [48]:
diff_dataset = pd.Series(diff_dataset)

In [49]:
diff_dataset.head()

0    262257
1        51
2        57
3        51
4        46
dtype: int64

In [50]:
import seaborn as sns
%matplotlib inline

In [51]:
diff_dataset.describe()

count    928991.000000
mean          0.298587
std         295.352490
min      -41416.000000
25%          -7.000000
50%           0.000000
75%           6.000000
max      262257.000000
dtype: float64

In [52]:
len(diff_dataset)

928991

In [57]:
((diff_dataset >= 64) | (diff_dataset <= -64)).sum() 

25563

In [58]:
25563/928991 * 100

2.7516951186825276

In [59]:
(2.7516951186825276 * 24 + (100 - 2.7516951186825276) *  8) / (100 * 20)


0.4220135609494602

In [60]:
bytesToSend = []
for i in diff_dataset :
    if(i >= 64 or i <= -64) :
        # Send 3 Bytes
        number = decimalToBinary(abs(i) , 20)
        if(i < 0):
            firstByte = '01' + number[0:6]
        else :
            firstByte = '00' + number[0:6]
        secondByte = '0' + number[6:13]
        thirdByte = '0' + number[13:]
        bytesToSend.append(firstByte)
        bytesToSend.append(secondByte)
        bytesToSend.append(thirdByte)
    else :
        number = decimalToBinary(abs(i) , 6)
        if(i < 0):
            firstByte = '11' + number
        else :
            firstByte = '10' + number
        bytesToSend.append(firstByte)

In [61]:
len(bytesToSend)

980117

# Code for Stream

In [62]:
import time
import struct

In [63]:
bytesToSend

['00010000',
 '00000000',
 '01110001',
 '10110011',
 '10111001',
 '10110011',
 '10101110',
 '10101001',
 '10100110',
 '10100010',
 '10011111',
 '10010100',
 '10010001',
 '10001111',
 '10001101',
 '10001100',
 '10001100',
 '10001010',
 '10000011',
 '11000001',
 '10000000',
 '11000001',
 '10000000',
 '11000001',
 '10000000',
 '10000010',
 '10000010',
 '01000000',
 '00000000',
 '01110011',
 '01000000',
 '00000000',
 '01101000',
 '01000000',
 '00000000',
 '01101000',
 '01000000',
 '00000000',
 '01011110',
 '01000000',
 '00000000',
 '01010101',
 '01000000',
 '00000000',
 '01001101',
 '01000000',
 '00000000',
 '01000101',
 '11111111',
 '11111001',
 '11101010',
 '11100110',
 '11100011',
 '11011111',
 '11011100',
 '11011001',
 '11010111',
 '11011100',
 '11011100',
 '11011010',
 '11010101',
 '11010101',
 '11010011',
 '11010000',
 '11010000',
 '11001110',
 '11001100',
 '11001011',
 '11001010',
 '11001001',
 '11001001',
 '11001010',
 '11001111',
 '11001111',
 '11001101',
 '11001100',
 '11001011',

In [64]:
binaryToDecimal('11111111')

255

In [65]:
bytesToSend[1990:2010]

['01000000',
 '00000000',
 '01010110',
 '01000000',
 '00000000',
 '01001110',
 '01000000',
 '00000000',
 '01000111',
 '01000000',
 '00000000',
 '01000000',
 '11111010',
 '11110101',
 '11101111',
 '11101011',
 '11100111',
 '11100011',
 '11100010',
 '11100110']

In [71]:
import socket
# host = 'localhost'  # as both code is running on same pc

host = 'localhost'
port = 3000  # socket server port number

In [72]:
def sendMessage(dataByte , socket):
    socket.send(dataByte)

In [73]:
def storeMessage(dataByte):
    message_array.append(dataByte)

In [74]:
def client_program(obv_factor):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # instantiate
    client_socket.connect((host, port))  # connect to the server

    count = 0
    message_array = []
    for i in range(0, int(len(bytesToSend) * obv_factor)):
        final = binaryToDecimal(bytesToSend[i]).to_bytes(1 ,  byteorder='little')
        message_array.append(final)
        count += 1

    for i in message_array :
        client_socket.send(i)

    print("Packets Transferred : " , count)    
    client_socket.send(''.encode())
    client_socket.close()  # close the connection

In [75]:
client_program(1)

Packets Transferred :  980117


In [76]:
observations = [1.0, 0.5, 0.25, 0.125, 0.0625]

for i in observations :
    for j in range(3) :
        time.sleep(1)
        message_array = []
        client_program(i)
        time.sleep(5)

Packets Transferred :  980117
Packets Transferred :  980117
Packets Transferred :  980117
Packets Transferred :  490058
Packets Transferred :  490058
Packets Transferred :  490058
Packets Transferred :  245029
Packets Transferred :  245029
Packets Transferred :  245029
Packets Transferred :  122514
Packets Transferred :  122514
Packets Transferred :  122514
Packets Transferred :  61257
Packets Transferred :  61257
Packets Transferred :  61257


# Code for Un-compressed

In [77]:
UCbytesToSend = []
for i in int_dataset :
    number = decimalToBinary(abs(i) , 24)
    UCbytesToSend.append(number)

In [78]:
len(UCbytesToSend)

928991

In [79]:
def UC_client_program(obv_factor):
    host = 'localhost'
    port = 6000
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # instantiate
    client_socket.connect((host, port))  # connect to the server

    count = 0
    message_array = []
    for i in range(0, int(len(UCbytesToSend) * obv_factor)):
        final = binaryToDecimal(UCbytesToSend[i]).to_bytes(3, byteorder="little")
        message_array.append(final)
        count += 1

    for i in message_array :
        client_socket.send(i)

    print("Packets Transferred : " , count)    
    client_socket.send(''.encode())
    client_socket.close()  # close the connection

In [80]:
UC_client_program(1)

Packets Transferred :  928991


In [81]:
observations = [1.0, 0.5, 0.25, 0.125, 0.0625]

for i in observations :
    for j in range(3) :
        time.sleep(1)
        message_array = []
        UC_client_program(i)
        time.sleep(5)

Packets Transferred :  928991
Packets Transferred :  928991
Packets Transferred :  928991
Packets Transferred :  464495
Packets Transferred :  464495
Packets Transferred :  464495
Packets Transferred :  232247
Packets Transferred :  232247
Packets Transferred :  232247
Packets Transferred :  116123
Packets Transferred :  116123
Packets Transferred :  116123
Packets Transferred :  58061
Packets Transferred :  58061
Packets Transferred :  58061
